In [1]:
import sys
import torch

# add the unet helpers
sys.path.append('../../')
from test_unet_helpers import *
from unet_models import unet11_MS
from utils import variable

from gbdxtools import CatalogImage

/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expe

In [ ]:
## write out with gdal
import gdal, osr

def array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array):

    cols = array.shape[1]
    rows = array.shape[0]
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Float64)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(4326)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

# RGB-NIR
model_paths = ['./files_PS_bgrn1/bgrn1_ps_ep75_step1575_b24.pt']

dg_scene_id = '1030010057062200' #2016
dg_scene_id = '104001000AA4B500' #2015
img_2m = CatalogImage(dg_scene_id, pansharpened=True, acomp=True)
chips = img_2m.window_cover((512,512), pad=False)
    
# 2-19-2019 using batch 32 RGB-NIR model
for model_path in model_paths:
    
    torch.cuda.empty_cache()
    
    #model = unet11(pretrained=False) # B-G-R
    model = unet11_MS(num_bands=4, pretrained=False) 
    model.eval()
        
    # load on CPU
    if os.path.exists(model_path):
        state_dict = torch.load(str(model_path), map_location='cpu')
        epoch = state_dict['epoch']
        step = state_dict['step']
        new_state_dict = OrderedDict()
        for k, v in state_dict['model'].items():
            name = k[7:] # remove 'module.' of dataparallel
            new_state_dict[name]=v

        model.load_state_dict(new_state_dict)
        print('Restored model, epoch {}, step {:,}'.format(epoch, step))

    
    # iterate over the image windows, getting the 4 bands
    for i,chip in enumerate(chips):
        
        rname = './rastertiles2015/denver_2015_t{0:0>6}_{1}.tif'.format(i+1, dg_scene_id)
        if os.path.exists(rname):
            continue
        
        print('on {} of {}'.format(i+1, 27051))
        img_arr = chip[[1,2,4,6], :, :].compute()/10000 #B-G-R-N1
        img_arr = torch.from_numpy(img_arr)
        #img_arr = img_transform(torch.from_numpy(img_arr))
        
        #input_img = torch.unsqueeze(variable(img_arr, volatile=True), dim=0)
        input_img = torch.unsqueeze(variable(img_arr, volatile=True), dim=0).cpu()
        del img_arr

        # model inference
        big_mask = model(input_img)
        #print(big_mask.cpu().detach().numpy().squeeze().max())

        # file creation
        #raster_id = os.path.basename(model_path).split('.')[0]
        #rname = '../tiles_64x64/denver_2016_t{0:0>6}_{1}.tif'.format(i+1, dg_scene_id)
        aff = chip.affine
        raster_origin = (aff.c, aff.f)
        pixel_height = aff.e
        pixel_width = aff.a
        array2raster(rname, raster_origin, pixel_width, pixel_height, big_mask.cpu().detach().numpy().squeeze())

        del input_img
        del big_mask
        

        torch.cuda.empty_cache()

num_bands is  4
Restored model, epoch files_PS_bgrn1/bgrn1_ps_ep75_step1575_b24.pt, step 1,575
on 1 of 27051


../../utils.py:16: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return cuda(Variable(x, volatile=volatile))


on 2 of 27051
on 3 of 27051
on 4 of 27051
on 5 of 27051
on 6 of 27051
on 7 of 27051
on 8 of 27051
on 9 of 27051
on 10 of 27051
on 11 of 27051
on 12 of 27051
on 13 of 27051
on 14 of 27051
on 15 of 27051
on 16 of 27051
on 17 of 27051
on 18 of 27051
on 19 of 27051
on 20 of 27051
on 21 of 27051
on 22 of 27051
on 23 of 27051
on 24 of 27051
on 25 of 27051
on 26 of 27051
on 27 of 27051
on 28 of 27051
on 29 of 27051
on 30 of 27051
on 31 of 27051
on 32 of 27051
on 33 of 27051
on 34 of 27051
on 35 of 27051
on 36 of 27051
on 37 of 27051
on 38 of 27051
on 39 of 27051
on 40 of 27051
on 41 of 27051
on 42 of 27051
on 43 of 27051
on 44 of 27051
on 45 of 27051
on 46 of 27051
on 47 of 27051
on 48 of 27051
on 49 of 27051
on 50 of 27051
on 51 of 27051
on 52 of 27051
on 53 of 27051
on 54 of 27051
on 55 of 27051
on 56 of 27051
on 57 of 27051
on 58 of 27051
on 59 of 27051
on 60 of 27051
on 61 of 27051
on 62 of 27051
on 63 of 27051
on 64 of 27051
on 65 of 27051
on 66 of 27051
on 67 of 27051
on 68 of 27051
on 

In [ ]:
len(chips)